In [4]:
import moviepy.editor as mpe
import whisper_timestamped as whisper
import json
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import random
import os

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [5]:
def get_file_names(folder, file_type='.mp4'):
    folder_path = folder
    file_names = []
    
    for file in os.listdir(folder_path):
        if file.endswith(file_type):
            file_name = os.path.splitext(os.path.basename(file))[0]
            file_names.append(file_name)
    
    return file_names

def make_transcription(audio_filename):
    audio = whisper.load_audio(audio_filename)
    # model = whisper.load_model("tiny", device="gpu")
    model = whisper.load_model('base')
    result = whisper.transcribe(model, audio, language="en")
    return result

def make_timestamps(transcription):
    timestamps = []
    for seg in transcription['segments']:
        for word in seg['words']:
            timestamps.append([(word['start'], word['end']), word['text']])
    return [timestamps, transcription]

# Function to create a TextClip for each subtitle
def make_textclip(sub, font_size, font_color):
    start, end = sub[0]
    txt = sub[1]
    return [TextClip(txt, fontsize=font_size, font='AvenirNext-Bold', color=font_color, stroke_color='black', stroke_width=3), start, end]

def resize(t, duration):
    # Starting scale factor
    duration = min(duration, 0.2)
    start_scale = 1
    # End scale factor (the size to which the text should grow)
    end_scale = 1.2
    # Calculate the scaling factor based on elapsed time and total duration
    t * start_scale + ((duration - t) / duration) ** 0.3 * (end_scale - start_scale)
    scale_factor = min(start_scale + t/duration * (end_scale - start_scale), end_scale)
    return scale_factor

# Function to overlay the text clips on the video
def overlay_video(video_filename, audio_data, animate=False):
    [subtitles, transcription] = audio_data
    # Create a list of TextClips
    text_clips = []
    if animate:
        for sub in subtitles:
            color = 'rgb(126, 217, 87)' if (random.choice(range(10)) > 7) else 'white'
            color = 'yellow' if (random.choice(range(10)) > 6) else color
            [clip, start, end] = make_textclip(sub, font_size=65, font_color=color)
            duration = end - start
            resize_clip = clip.resize(lambda t: resize(t, duration))
            new_clip = resize_clip.set_start(start).set_duration(end - start).set_position('center')
            text_clips.append(new_clip)
    else:
        for sub in subtitles:
            #color = '#FF7ED957' if (random.choice(range(10)) >= 4) else 'white'
            [clip, start, end] = make_textclip(sub, 70, color)
            new_clip = clip.set_start(start).set_duration(end - start).set_position('center')
            text_clips.append(new_clip)
    # Overlay the text on the video
    video = mpe.VideoFileClip(video_filename)
    overlay = mpe.CompositeVideoClip([video, *text_clips])
    video_duration = overlay.duration
    logo = (mpe.ImageClip("../videos/assets/watermark.png")
          .set_duration(video_duration)
          .resize(height=150) # if you need to resize...
          #.margin(right=8, top=8, opacity=.5) # (optional) logo-border padding
          .set_pos(("left","bottom")))
    video_h = overlay.h
    video_w = overlay.w
    end_card = mpe.VideoFileClip("../videos/assets/EndCard.mp4").resize(height=video_h, width=video_w).set_start(video_duration)
    final = mpe.CompositeVideoClip([overlay, logo, end_card])
    return final

In [22]:
TEST_ID = '7nuedz2'
TEST_ID2 = '75p8dn'
TEST_ID3 = '8zlzuy'
TEST_ID4 = 'd7qi0y'
TEST_ID5 = 'mhkbe3'
TEST_ID6 = '85kaz5'

ID = '7jpnjl'

transcription = make_transcription(f'../videos/raw_videos2/{ID}.mp4', )
# Define the text and timestamps
subtitles = make_timestamps(transcription)
final = overlay_video(f"../videos/raw_videos2/{ID}.mp4", subtitles, animate=True)
final.write_videofile(f"../videos/tests/{ID}.mp4", codec="libx264", audio_codec='aac')

100%|██████████| 4462/4462 [00:16<00:00, 271.62frames/s]


Moviepy - Building video ../videos/tests/7jpnjl.mp4.
MoviePy - Writing audio in 7jpnjlTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../videos/tests/7jpnjl.mp4



Moviepy - Done !
Moviepy - video ready ../videos/tests/7jpnjl.mp4


In [6]:
script_id = 'm98m4q'
file_path = f'../videos/processed_videos2/{script_id}.mp4'
transcription = make_transcription(f'../videos/tests/testing_formula.mp4', )
# Define the text and timestamps
subtitles = make_timestamps(transcription)
final = overlay_video(f'../videos/tests/testing_formula.mp4', subtitles, animate=True)
final.write_videofile(f"../videos/tests/{script_id}.mp4", codec="libx264", audio_codec='aac')

100%|██████████| 5493/5493 [00:18<00:00, 304.18frames/s]


Moviepy - Building video ../videos/tests/m98m4q.mp4.
MoviePy - Writing audio in m98m4qTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../videos/tests/m98m4q.mp4



Moviepy - Done !
Moviepy - video ready ../videos/tests/m98m4q.mp4


In [3]:
script_names = get_file_names('../videos/raw_videos2', '.mp4')
for script_id in script_names:
    file_path = f'../videos/processed_videos2/{script_id}.mp4'
    if not os.path.exists(file_path ):
        transcription = make_transcription(f'../videos/raw_videos2/{script_id}.mp4', )
        # Define the text and timestamps
        subtitles = make_timestamps(transcription)
        final = overlay_video(f"../videos/raw_videos2/{script_id}.mp4", subtitles, animate=True)
        final.write_videofile(f"../videos/processed_videos2/{script_id}.mp4", codec="libx264", audio_codec='aac')
    else:
        print(f'{script_id} already exists.')

7jpnjl already exists.


100%|██████████| 4462/4462 [00:14<00:00, 300.48frames/s]


Moviepy - Building video ../videos/processed_videos2/7jpnjl_1.mp4.
MoviePy - Writing audio in 7jpnjl_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../videos/processed_videos2/7jpnjl_1.mp4



Moviepy - Done !
Moviepy - video ready ../videos/processed_videos2/7jpnjl_1.mp4


...

Moviepy - Done !
Moviepy - video ready ../videos/processed_videos2/908y52.mp4


100%|██████████| 4699/4699 [00:14<00:00, 315.30frames/s]


Moviepy - Building video ../videos/processed_videos2/bmzivx.mp4.
MoviePy - Writing audio in bmzivxTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../videos/processed_videos2/bmzivx.mp4



Moviepy - Done !
Moviepy - video ready ../videos/processed_videos2/bmzivx.mp4


In [25]:
script_names

['7jpnjl', '7jpnjl_1', '7sewfa', '7xmrjn', '7yx524', '908y52', 'bmzivx']

In [13]:
ID = '85kaz5'
transcription = make_transcription(f'../videos/raw_videos/{ID}.mp4')
# Define the text and timestamps
subtitles = make_timestamps(transcription)

100%|██████████| 5967/5967 [00:19<00:00, 302.50frames/s]


In [45]:
items = []
time = 0
width = 1920
height = 1080
for sub in subtitles[0:20]:
    [clip, start, end] = make_textclip(sub, font_size=100, font_color='white')
    duration = end - start
    resize_clip = clip.resize(lambda t: resize(t, duration))
    new_clip = resize_clip.set_start(start).set_duration(end - start).set_position('center')
    time += duration
    items.append(new_clip)

bg_clip = ColorClip(size=(width, height), color=(0,0,0)).set_duration(time)

In [36]:
time

5.22

In [46]:
video = CompositeVideoClip([bg_clip] + items)

# 6. Export the Final Video
# Save the composite video to a file with a frame rate of 60 fps
video.write_videofile("scaling_text_1080p.mp4", fps=60)

                                                             


                                                    




                                              






                                        







                                      

                                                       
                                                          



                                                 





                                           








t:   6%|▌         | 27/441 [19:36<00:15, 26.76it/s, now=None] 



































                                                             


                                                    




                                              






                                        







                                      

                                                       
                                                          



                                   

Moviepy - Building video scaling_text_1080p.mp4.
Moviepy - Writing video scaling_text_1080p.mp4



Moviepy - Done !
Moviepy - video ready scaling_text_1080p.mp4
